In [63]:
import datetime as dt
import time
import logging as log
from multiprocessing import Pool, TimeoutError

%run BreakSectionManager.ipynb
%run Util.ipynb
%run Constant.ipynb
%run Metric.ipynb
%run Operation.ipynb
%run Account.ipynb
%run Policy_Loader.ipynb
%run Evaluation.ipynb


class RocketRegression:
    
    def __init__(self, policy_name):
        self.policy_name = policy_name
        self.policy_loader = Policy_Loader(self.policy_name)
        self.debug = self.policy_loader.enabled_log_debug()
        self.context = self.policy_loader.get_context()
        
        self.level = self.context['level']
        self.short = self.context['short']
        self.mid = self.context['mid']
        
    def build_operation_file_path(self,symbol,regression_end_date):
        return report_dir+'operations_'+symbol+'_'+str(self.short)+'_'+str(self.mid)+'_'+self.level+'_'+regression_end_date+'_'+operation_version+'.csv'
    
    def build_deal_file_path(self,symbol,regression_end_date):
        return report_dir+'deals_'+symbol+'_'+str(self.short)+'_'+str(self.mid)+'_'+self.level+'_'+regression_end_date+'_'+operation_version+'.csv'
        
    def build_asset_file_path(self,symbol,regression_end_date):
        return report_dir+'account_asset_'+symbol+'_'+str(self.short)+'_'+str(self.mid)+'_'+self.level+'_'+regression_end_date+'_'+operation_version+'.csv'
    
    def persist(self,account,operation_df,symbol,regression_end_date):
        account.get_asset_df().to_csv(self.build_asset_file_path(symbol,regression_end_date),index=False)
        account.deal_df.to_csv(self.build_deal_file_path(symbol,regression_end_date),index=False)
        operation_df.to_csv(self.build_operation_file_path(symbol,regression_end_date), index=False)
        
    
    def start_regression_on_one_stock(self,symbol,regression_end_date):
        start = time.time()
        sectionManager = BreakSectionManager(symbol,self.context)
        section_df = sectionManager.get_all_break_sections()
        section_df = section_df[section_df['d_s_datetime']<to_datetime(regression_end_date)]
        
        metric = Metric(symbol,self.context)
        
        stock_start_date = metric.get_stock_start_date()
        stock_end_date = metric.get_stock_end_date()
        log.info('stock_start_date ' + stock_start_date+' stock_end_date:'+stock_end_date)
        
        account = Account(stock_start_date,stock_end_date,self.context,self.policy_name)
        operation = Operation(account,symbol,self.context,self.policy_name)
        
        for key,row in section_df.iterrows():
            start_date = row['d_s_date']
            end_date = row['d_e_date']
            
            start_datetime = to_datetime(start_date)
            end_datetime = to_datetime(end_date)

            buy_price = metric.get_cur_price(start_date)
            
            if(self.debug==True):
                print('\n'+row['symbol']+' section '+start_date+'===>'+end_date)
            while(start_datetime<=end_datetime):
                cur_date = start_datetime.strftime(YMD_format)
                
                if(metric.is_today_open(cur_date)):
                    account.daily_audit(cur_date)

                    if(account.can_open_new_stock()==True):
                        if(operation.is_buy_point(cur_date)==True):
                            operation.open_opsition(cur_date)

                    if(account.has_shares()==True):
                        if(operation.is_sell_point(cur_date)==True):
                            operation.sell_stock(cur_date)

                start_datetime += dt.timedelta(days = 1)
                
        operation_df = operation.get_operations()
        self.persist(account,operation_df,symbol,regression_end_date)
        
        evaluation = Evaluation(account,self.context,self.policy_name)
        evaluation_df = evaluation.get_evaluation_report(symbol,stock_start_date,stock_end_date,
                                                         operation_df)

        end = time.time()
        if(self.debug==False):
            print(symbol+' regression cost time ' + str(round(end-start,1))+' seconds')
        return evaluation_df
    
    
    def start_sync_regression(self,scale,regression_end_date):
        evaluation_df = pd.DataFrame(columns=evaluation_columns)
        symbols = get_symbols(scale,home)
        total = len(symbols) 
        
        for symbol in symbols:
            one_evaluation = self.start_regression_on_one_stock(symbol,regression_end_date)
            evaluation_df = pd.concat([one_evaluation,evaluation_df], ignore_index=True, sort=False)
           
        
        evaluation_df['loss_times'] =evaluation_df['loss_times'].astype('int')
        evaluation_df['deal_count'] =evaluation_df['deal_count'].astype('int')
        evaluation_df['r2'] =evaluation_df['r2'].astype('float')
        evaluation_df['rar'] =evaluation_df['rar'].astype('float')

        return evaluation_df
    
    def start_regression(self,scale,regression_end_date,process_num):
        print('context:' + str(self.context))
        
        if(process_num==1):
            return self.start_sync_regression(scale,regression_end_date)
        
        evaluation_df = pd.DataFrame(columns=evaluation_columns)
        symbols = get_symbols(scale,home)
        total = len(symbols) 
        
        pool = Pool(processes=process_num)
        tasks = []
        for symbol in symbols:
            task = pool.apply_async(self.start_regression_on_one_stock, (symbol,regression_end_date))
            tasks.append(task)
            
        for task in tasks:
            try:
                one_evaluation = task.get()
                evaluation_df = pd.concat([one_evaluation,evaluation_df], ignore_index=True, sort=False)
            except Exception as e:
                print(str(e))
        
        pool.close()
        pool.join()
        
        evaluation_df['loss_times'] =evaluation_df['loss_times'].astype('int')
        evaluation_df['deal_count'] =evaluation_df['deal_count'].astype('int')
        evaluation_df['r2'] =evaluation_df['r2'].astype('float')
        evaluation_df['rar'] =evaluation_df['rar'].astype('float')

        return evaluation_df
    

In [64]:
regression = RocketRegression('rocket_break_v3_enable_adding_position_based_percent.json')

regression_end='2019-05-30'
scale=300
process_num=24

evaluation_df = regression.start_regression(scale,regression_end,process_num)
evaluation_df

context:{'short': 20, 'mid': 240, 'scale_start': 0, 'scale_end': 3, 'level': 'Day'}


/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


SZ#002343 regression cost time 15.2 seconds
SZ#002345 regression cost time 16.1 seconds
SZ#002505 regression cost time 16.1 seconds


/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


SZ#002181 regression cost time 19.9 seconds
SZ#002183 regression cost time 22.1 seconds
SZ#002182 regression cost time 27.1 seconds


...

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:81: RuntimeWarning: divide by zero encountered in double_scalars


SZ#002544 regression cost time 24.6 seconds
SZ#000625 regression cost time 55.1 seconds
SZ#000626 regression cost time 46.3 seconds
SZ#002546 regression cost time 11.6 seconds
SZ#002221 regression cost time 27.7 seconds
SZ#000868 regression cost time 37.6 seconds
SZ#000875 regression cost time 24.4 seconds
SZ#000046 regression cost time 52.7 seconds
SZ#002060 regression cost time 31.2 seconds
SZ#000627 regression cost time 42.1 seconds
SZ#002385 regression cost time 12.9 seconds
SZ#000628 regression cost time 36.4 seconds


/opt/conda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/conda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


SZ#000048 regression cost time 46.2 seconds
SZ#002547 regression cost time 14.1 seconds
SZ#002222 regression cost time 24.4 seconds
SZ#002062 regression cost time 19.2 seconds
SZ#002384 regression cost time 21.7 seconds
SZ#002061 regression cost time 29.0 seconds
SZ#000049 regression cost time 40.2 seconds
SZ#000629 regression cost time 35.5 seconds
SZ#000869 regression cost time 41.0 seconds
SZ#002223 regression cost time 21.7 seconds
SZ#000050 regression cost time 37.9 seconds
SZ#000055 regression cost time 28.0 seconds
SZ#000630 regression cost time 30.3 seconds
cannot convert float infinity to integer


,symbol,start_date,end_date,policy_name,short,mid,max_drawdown,avg_year_return,loss_times,year_regular_return,...,std_month_return,profit_month_rate,win_rate,mar,sharp,rar,avg_drawdown,avg_drawdown_days,r2,deal_count
0,SZ#002547,2011-02-18,2019-02-18,rocket_break_v3_enable_adding_position_based_p...,20,240,25.15,-0.35,2,-2.60,...,6.125791,60.00,66.67,-0.01,-1.804220,NaN,18.073633,48.000000,NaN,6
1,SZ#002385,2010-04-09,2019-02-18,rocket_break_v3_enable_adding_position_based_p...,20,240,15.75,-0.08,5,-0.56,...,4.296907,28.57,37.50,-0.01,-4.213481,NaN,13.983257,NaN,NaN,8
2,SZ#002223,2008-04-18,2019-02-18,rocket_break_v3_enable_adding_position_based_p...,20,240,28.43,4.75,3,21.81,...,6.947662,58.82,50.00,0.17,-0.654229,105.000000,24.009416,308.000000,5.182626,12
3,SZ#002062,2006-08-16,2019-02-18,rocket_break_v3_enable_adding_position_based_p...,20,240,32.35,-0.06,7,-0.33,...,9.847879,33.33,38.46,-0.00,-4.618941,-70.000000,27.369425,246.000000,-3.794811,13
4,SZ#000630,2000-02-14,2019-02-18,rocket_break_v3_enable_adding_position_based_p...,20,240,17.81,7.70,3,49.91,...,4.857548,58.97,52.94,0.43,0.663551,87.500000,13.601410,93.333333,25.158238,17
5,SZ#000055,2000-02-14,2019-02-18,rocket_break_v3_enable_adding_position_based_p...,20,240,18.68,2.23,3,11.54,...,3.468991,55.26,50.00,0.12,-2.635346,-29.000000,14.463348,182.000000,-4.021153,16
6,SZ#002546,2011-02-11,2019-02-18,rocket_break_v3_enable_adding_position_based_p...,20,240,16.78,9.87,2,77.23,...,7.353325,60.00,50.00,0.59,0.313275,NaN,16.301641,32.000000,NaN,6
7,SZ#002384,2010-04-09,2019-02-18,rocket_break_v3_enable_adding_position_based_p...,20,240,18.03,8.56,4,32.39,...,6.803106,43.48,20.00,0.47,1.431686,NaN,13.606870,89.500000,NaN,10
8,SZ#002222,2008-03-19,2019-02-18,rocket_break_v3_enable_adding_position_based_p...,20,240,23.61,3.04,3,25.76,...,7.694192,57.69,45.45,0.13,-0.448861,-27.000000,14.323916,79.500000,-8.654216,11
9,SZ#002061,2006-08-16,2019-02-18,rocket_break_v3_enable_adding_position_based_p...,20,240,30.30,10.41,4,54.38,...,9.410440,45.16,46.15,0.34,0.657117,283.000000,25.307249,269.000000,15.173371,13


In [66]:
evaluation_df.describe()

,max_drawdown,avg_year_return,loss_times,year_regular_return,mean_year_return,std_year_return,mean_month_return,std_month_return,profit_month_rate,win_rate,mar,sharp,rar,avg_drawdown,avg_drawdown_days,r2,deal_count
count,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,288.000000,232.000000,288.000000,228.000000,186.000000,288.00000
mean,20.517188,9.914271,3.319444,59.062222,4.653973,12.833225,1.388294,7.715900,55.544861,47.362014,0.593021,-0.314507,-27.547610,16.391204,148.350000,-23.414786,13.28125
std,7.704989,14.206866,1.709383,83.788440,4.697619,5.795595,1.704873,5.056206,10.146556,13.554262,0.862918,2.911242,635.171132,6.215662,95.625696,425.395824,5.46073
min,5.770000,-5.350000,0.000000,-33.570000,-7.936000,3.443434,-1.567419,1.667765,11.110000,18.180000,-0.200000,-26.162951,-8190.000000,3.835997,3.000000,-5617.160383,4.00000
25%,15.772500,2.450000,2.000000,14.125000,1.620813,8.693870,0.275529,4.855451,50.000000,38.460000,0.107500,-0.792480,-103.500000,12.402643,87.375000,-18.481461,9.00000
50%,19.715000,6.810000,3.000000,37.220000,4.367440,11.842771,1.011672,6.379117,55.650000,47.060000,0.350000,0.157608,15.630952,15.131817,126.000000,2.248485,13.00000
75%,23.902500,12.707500,4.000000,78.047500,7.186562,16.030182,2.017081,8.464702,62.790000,54.550000,0.772500,0.884046,138.464286,19.422329,183.750000,18.839270,17.00000
max,82.500000,144.600000,11.000000,673.210000,28.846667,38.920729,11.769286,47.451642,77.780000,100.000000,8.180000,13.310245,1300.000000,60.597018,522.000000,713.822922,29.00000


In [ ]:
print('Helo')